In [1]:
import cv2
import numpy as np
import math
cap=cv2.VideoCapture('test.mp4')

def get_lines_edges(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    blurred=cv2.GaussianBlur(gray,(5,5),0)
    edges=cv2.Canny(blurred,100,150)
    return edges
def get_lines(img):
    lines=cv2.HoughLinesP(img,2,np.pi/180.0,100,np.array([]),minLineLength=40,maxLineGap=15)
    return lines
def is_right(x1,x2,w):
    return (x1+x2)/2 >w/2
def average_lines(lines):
    right_fit=[]
    left_fit=[]
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            fit=np.polyfit((x1,x2),(y1,y2),1)
            
            if fit[0]>0:#slope
                right_fit.append(fit)
            else:
                left_fit.append(fit)
    right_avg=np.average(right_fit,axis=0)
    left_avg=np.average(left_fit,axis=0)
    return right_avg,left_avg
def make_points(img,fit):
    m,c=fit
    y1=int(img.shape[0])
    y2=int((img.shape[0]*1)/8)
    x1=(y1-c)//m
    x2=(y2-c)//m
    return int(x1),int(y1),int(x2),int(y2)

def display_lines(img,lines):
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            cv2.line(img,(x1,y1),(x2,y2),(255,0,0),2)
def range_of(x1,x2,a):
    return x1+a>=x2 and x1-a<=x2
while cap.isOpened():
    ret,frame=cap.read()
    if ret:
        frame=cv2.resize(frame,(700,600))
        pts=np.float32([(0,frame.shape[0]),(200,frame.shape[0]-180),(500,frame.shape[0]-180),(700,frame.shape[0])])
        train_p = np.float32([[0, 400], [0, 0], [500, 0], [500, 400]])
        M=cv2.getPerspectiveTransform(pts,train_p)
        M_inv=cv2.getPerspectiveTransform(train_p,pts)
        img_new=cv2.warpPerspective(frame,M,(500,400))
        edges=get_lines_edges(img_new)
        
        lines=get_lines(edges)
        
        #display_lines(frame,lines)
        try:
            right_avg,left_avg=average_lines(lines)
            slope_r=round(right_avg[0],2)
            slope_l=round(left_avg[0],2)            
            x1_r,y1_r,x2_r,y2_r=make_points(img_new,right_avg)
            x1_l,y1_l,x2_l,y2_l=make_points(img_new,left_avg)
            
            x1=(x1_l+x1_r)//2
            x2=(x2_l+x2_r)//2
            
            cv2.line(img_new,(x1_r,y1_r),(x2_r,y2_r),(255,0,0),2)
            cv2.line(img_new,(x1_l,y1_l),(x2_l,y2_l),(0,0,255),2)
            cv2.line(img_new,(x1,y1_l),(x2,y2_l),(0,255,0),2)
            ang=(abs(round(math.degrees(math.atan(slope_r)),2))+abs(round(math.degrees(math.atan(slope_l)),2)))//2#average angle
            
            
            #if(range_of(x1,x2,15)):
                #cv2.putText(frame,"straight",(20,90),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
            #elif(x1<x2):
            #    cv2.putText(frame,"right",(20,90),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
            #    cv2.putText(frame,f"angle= {ang}",(20,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))

            #else:
            #   cv2.putText(frame,"left",(20,90),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))
            #    cv2.putText(frame,f"angle= {ang}",(20,30),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0))

                
            points_to_be_transformed = np.array([[[x1_r,y1_r],[x2_r,y2_r],[x1_l,y1_l],[x2_l,y2_l]]], dtype=np.float32)#to draw on original frame

            transformed_points=cv2.perspectiveTransform(points_to_be_transformed,M_inv)
            x1_r,y1_r=int(transformed_points[0][0][0]),int(transformed_points[0][0][1])
            x2_r,y2_r=int(transformed_points[0][1][0]),int(transformed_points[0][1][1])
            x1_l,y1_l=int(transformed_points[0][2][0]),int(transformed_points[0][2][1])
            x2_l,y2_l=int(transformed_points[0][3][0]),int(transformed_points[0][3][1])
            cv2.line(frame,(x1_l,y1_l),(x2_l,y2_l),(0,0,255),2)
            cv2.line(frame,(x1_r,y1_r),(x2_r,y2_r),(255,0,0),2)
            
        except:
            pass

        
        cv2.imshow('img_new',img_new)
        cv2.imshow('roi',edges)    
        
        
        
        
        cv2.imshow('frame',frame)
        key=cv2.waitKey(5)
        if key &0xff==ord('q'):
            break
    else:
        cap=cv2.VideoCapture('test.mp4')
        
cv2.destroyAllWindows()
        

C:\Users\abdel\anaconda3\envs\tfgpu\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\abdel\anaconda3\envs\tfgpu\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
